In [1]:
import os

import pandas as pd
import numpy as np

In [2]:
os.getcwd()

'/SAN/intelsys/rclearn/when-do-reading-comprehension-models-learn/notebooks/exploration'

In [3]:
os.chdir('../..')

In [4]:
fp = 'predictions/on-train/albert-xlarge-v2-squadv1-adversarialall-wu=100-lr=3e5-bs=32-msl=384-seed=27/checkpoint-7948/predict_predictions.json'

eval_predictions_df = pd.read_json(fp, orient='index')
eval_predictions_df.reset_index(inplace=True)
eval_predictions_df.rename(columns={'index': 'id', 0: "prediction_text"}, inplace=True)

In [5]:
eval_predictions_df.head()

,id,prediction_text
0,5733be284776f41900661182,Saint Bernadette Soubirous
1,5733be284776f4190066117f,a copper statue of Christ
2,5733be284776f41900661180,the Main Building
3,5733be284776f41900661181,a Marian place of prayer and reflection
4,5733be284776f4190066117e,a golden statue of the Virgin Mary


In [6]:
eval_predictions_df.shape

(117564, 2)

In [7]:
117564 - 87599

29965

In [8]:
from datasets import load_dataset, load_metric

In [9]:
squad_v1_train = load_dataset('squad', split='train')
adversarial_qa_train = load_dataset('adversarial_qa', 'adversarialQA')['train']

Reusing dataset squad (/home/sgeorge/.cache/huggingface/datasets/squad/plain_text/1.0.0/6b6c4172d0119c74515f44ea0b8262efe4897f2ddb6613e5e915840fdc309c16)
Reusing dataset adversarial_qa (/home/sgeorge/.cache/huggingface/datasets/adversarial_qa/adversarialQA/1.0.0/92356be07b087c5c6a543138757828b8d61ca34de8a87807d40bbc0e6c68f04b)


In [10]:
squad_v1_train_df = pd.DataFrame(squad_v1_train)
print(squad_v1_train_df.shape)
squad_v1_train_df.head()

(87599, 5)


,answers,context,id,question,title
0,"{'text': ['Saint Bernadette Soubirous'], 'answ...","Architecturally, the school has a Catholic cha...",5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,University_of_Notre_Dame
1,"{'text': ['a copper statue of Christ'], 'answe...","Architecturally, the school has a Catholic cha...",5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,University_of_Notre_Dame
2,"{'text': ['the Main Building'], 'answer_start'...","Architecturally, the school has a Catholic cha...",5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,University_of_Notre_Dame
3,{'text': ['a Marian place of prayer and reflec...,"Architecturally, the school has a Catholic cha...",5733be284776f41900661181,What is the Grotto at Notre Dame?,University_of_Notre_Dame
4,{'text': ['a golden statue of the Virgin Mary'...,"Architecturally, the school has a Catholic cha...",5733be284776f4190066117e,What sits on top of the Main Building at Notre...,University_of_Notre_Dame


In [12]:
adversarial_qa_train_df = pd.DataFrame(adversarial_qa_train)
print(adversarial_qa_train_df.shape)
adversarial_qa_train_df.head()

(30000, 6)


,answers,context,id,metadata,question,title
0,"{'text': ['isolated from the bloodstream'], 'a...",Another approach to brain function is to exami...,7ba1e8f4261d3170fcf42e84a81dd749116fae95,"{'split': 'train', 'model_in_the_loop': 'Combi...",What sare the benifts of the blood brain barrir?,Brain
1,"{'text': ['brain'], 'answer_start': [280]}",Another approach to brain function is to exami...,5ec5ef305a259311596e85d811ade30bd68b079d,"{'split': 'train', 'model_in_the_loop': 'Combi...",What is surrounded by cerebrospinal fluid?,Brain
2,"{'text': ['brain'], 'answer_start': [280]}",Another approach to brain function is to exami...,7cb230edfb15ad1fda8d157af1f2b574cbb02b4c,"{'split': 'train', 'model_in_the_loop': 'Combi...",What does the skull protect?,Brain
3,"{'text': ['chemicals'], 'answer_start': [723]}",Another approach to brain function is to exami...,e1850f2a48b8f7c2231cec41ed63c1b638a8e2c7,"{'split': 'train', 'model_in_the_loop': 'Combi...",What has been injected into rats to produce pr...,Brain
4,"{'text': ['brain damage'], 'answer_start': [409]}",Another approach to brain function is to exami...,7bc0ae1a8a24ea4f3398b5236ab9569bbc3e820b,"{'split': 'train', 'model_in_the_loop': 'Combi...",What can cause issues with how the brain works?,Brain


In [20]:
adversarial_qa_train_df['id'].apply(len).value_counts()

40    30000
Name: id, dtype: int64

In [19]:
eval_predictions_df['id'].apply(len).value_counts()

24    87599
40    29965
Name: id, dtype: int64

In [24]:
set(adversarial_qa_train_df['id']) - set(eval_predictions_df['id'])

set()

In [30]:
adversarial_qa_train_df[adversarial_qa_train_df['id'].duplicated()].head()

,answers,context,id,metadata,question,title
93,"{'text': ['Campaign'], 'answer_start': [996]}",Provide Campaign Assistance. Minority leaders ...,4d3cb5677211ee32895ca9c66dad04d7152254d4,"{'split': 'train', 'model_in_the_loop': 'Combi...","What does the second C in ""DCCC"" stand for?",Party_leaders_of_the_United_States_House_of_Re...
98,"{'text': ['Congressional'], 'answer_start': [9...",Provide Campaign Assistance. Minority leaders ...,6ace7f0a1d6ca12f1a82aa4b6ed9b880c4d22154,"{'split': 'train', 'model_in_the_loop': 'Combi...","What does the first C in ""DCCC"" stand for?",Party_leaders_of_the_United_States_House_of_Re...
567,"{'text': ['Soviet authorities'], 'answer_start...","On 6 August 1940, Estonia was annexed by the S...",624f30226a8faceda9717ad02c0d1f191c9ae09d,"{'split': 'train', 'model_in_the_loop': 'Combi...",what is the second to last group mentioned?,Estonia
572,"{'text': ['Estonia'], 'answer_start': [623]}","On 6 August 1940, Estonia was annexed by the S...",3d2501fbc9220aa407693fcedf05ed14f0d3b729,"{'split': 'train', 'model_in_the_loop': 'Combi...",what is the second to last nation mentioned?,Estonia
574,"{'text': ['thousands of ordinary people'], 'an...","On 6 August 1940, Estonia was annexed by the S...",5be45686c520b4a199fb72872450ed29b568de66,"{'split': 'train', 'model_in_the_loop': 'Combi...",what is the last group mentioned?,Estonia


In [31]:
adversarial_qa_train_df[adversarial_qa_train_df['id'] == '4d3cb5677211ee32895ca9c66dad04d7152254d4']

,answers,context,id,metadata,question,title
91,"{'text': ['Campaign'], 'answer_start': [996]}",Provide Campaign Assistance. Minority leaders ...,4d3cb5677211ee32895ca9c66dad04d7152254d4,"{'split': 'train', 'model_in_the_loop': 'Combi...","What does the second C in ""DCCC"" stand for?",Party_leaders_of_the_United_States_House_of_Re...
93,"{'text': ['Campaign'], 'answer_start': [996]}",Provide Campaign Assistance. Minority leaders ...,4d3cb5677211ee32895ca9c66dad04d7152254d4,"{'split': 'train', 'model_in_the_loop': 'Combi...","What does the second C in ""DCCC"" stand for?",Party_leaders_of_the_United_States_House_of_Re...


In [15]:
combined = adversarial_qa_train_df.merge(eval_predictions_df, on='id', how='inner')

In [16]:
combined.shape

(30000, 7)

In [29]:
combined[combined['id'].duplicated()]

,answers,context,id,metadata,question,title,prediction_text
92,"{'text': ['Campaign'], 'answer_start': [996]}",Provide Campaign Assistance. Minority leaders ...,4d3cb5677211ee32895ca9c66dad04d7152254d4,"{'split': 'train', 'model_in_the_loop': 'Combi...","What does the second C in ""DCCC"" stand for?",Party_leaders_of_the_United_States_House_of_Re...,Congressional Campaign
98,"{'text': ['Congressional'], 'answer_start': [9...",Provide Campaign Assistance. Minority leaders ...,6ace7f0a1d6ca12f1a82aa4b6ed9b880c4d22154,"{'split': 'train', 'model_in_the_loop': 'Combi...","What does the first C in ""DCCC"" stand for?",Party_leaders_of_the_United_States_House_of_Re...,Congressional
566,"{'text': ['Soviet authorities'], 'answer_start...","On 6 August 1940, Estonia was annexed by the S...",624f30226a8faceda9717ad02c0d1f191c9ae09d,"{'split': 'train', 'model_in_the_loop': 'Combi...",what is the second to last group mentioned?,Estonia,Soviet authorities
568,"{'text': ['Estonia'], 'answer_start': [623]}","On 6 August 1940, Estonia was annexed by the S...",3d2501fbc9220aa407693fcedf05ed14f0d3b729,"{'split': 'train', 'model_in_the_loop': 'Combi...",what is the second to last nation mentioned?,Estonia,Estonia
571,"{'text': ['thousands of ordinary people'], 'an...","On 6 August 1940, Estonia was annexed by the S...",5be45686c520b4a199fb72872450ed29b568de66,"{'split': 'train', 'model_in_the_loop': 'Combi...",what is the last group mentioned?,Estonia,thousands of ordinary people
780,{'text': ['The first British settlement in wha...,The first British settlement in what is now Te...,85b70117535390499a17fc9a46d24bff621b6c22,"{'split': 'train', 'model_in_the_loop': 'Combi...",What happened in the 56th year of the 18th cen...,Tennessee,The first British settlement in what is now Te...
782,{'text': ['Captain Raymond Demeré relinquished...,The first British settlement in what is now Te...,06f0fc11ac4afbd691e3318326f505c204f40bd4,"{'split': 'train', 'model_in_the_loop': 'Combi...",What happened in the 57th year of the 18th cen...,Tennessee,Captain Raymond Demeré relinquished command
873,"{'text': ['glitches'], 'answer_start': [459]}",The original model Famicom featured two game c...,d9020b86419c96def44f251dd4e5595a67647d9f,"{'split': 'train', 'model_in_the_loop': 'Combi...",What caused the system to freeze?,Nintendo_Entertainment_System,glitches within the hardware
2311,"{'text': ['comparing different documents, or ""...","When comparing different documents, or ""witnes...",e126e36072e28c671999df9763a6c582ec8b6d0d,"{'split': 'train', 'model_in_the_loop': 'Combi...",What is a variant reading?,Textual_criticism,"When comparing different documents, or ""witnes..."
3410,"{'text': ['DIANE'], 'answer_start': [401]}",While videoconferencing technology was initial...,6195f231e14123c1ae84d44c23d907b1d4936748,"{'split': 'train', 'model_in_the_loop': 'Combi...",What is the five-letter acronym of the initiat...,Videoconferencing,DIANE


In [17]:
combined.head()

,answers,context,id,metadata,question,title,prediction_text
0,"{'text': ['isolated from the bloodstream'], 'a...",Another approach to brain function is to exami...,7ba1e8f4261d3170fcf42e84a81dd749116fae95,"{'split': 'train', 'model_in_the_loop': 'Combi...",What sare the benifts of the blood brain barrir?,Brain,isolated from the bloodstream by the blood–bra...
1,"{'text': ['brain'], 'answer_start': [280]}",Another approach to brain function is to exami...,5ec5ef305a259311596e85d811ade30bd68b079d,"{'split': 'train', 'model_in_the_loop': 'Combi...",What is surrounded by cerebrospinal fluid?,Brain,the brain
2,"{'text': ['brain'], 'answer_start': [280]}",Another approach to brain function is to exami...,7cb230edfb15ad1fda8d157af1f2b574cbb02b4c,"{'split': 'train', 'model_in_the_loop': 'Combi...",What does the skull protect?,Brain,brain
3,"{'text': ['chemicals'], 'answer_start': [723]}",Another approach to brain function is to exami...,e1850f2a48b8f7c2231cec41ed63c1b638a8e2c7,"{'split': 'train', 'model_in_the_loop': 'Combi...",What has been injected into rats to produce pr...,Brain,chemicals
4,"{'text': ['brain damage'], 'answer_start': [409]}",Another approach to brain function is to exami...,7bc0ae1a8a24ea4f3398b5236ab9569bbc3e820b,"{'split': 'train', 'model_in_the_loop': 'Combi...",What can cause issues with how the brain works?,Brain,damage to specific brain areas


In [18]:
from src.analysis.utils import squad1_evaluation

In [19]:
squad1_evaluation(combined['id'], combined['prediction_text'], combined['answers'])

2021-08-07 23:43:48,120 - Removing /home/sgeorge/.cache/huggingface/metrics/squad/default/default_experiment-1-0.arrow


{'exact_match': 68.90333333333334, 'f1': 80.86977870456109}

In [4]:
[name for name in os.listdir('predictions/albert-xlarge-v2-squadv2-wu=100-lr=3e5-bs=32-msl=384-seed=27/{}') if os.path.isfile(name)]

FileNotFoundError: [Errno 2] No such file or directory: 'predictions/albert-xlarge-v2-squadv2-wu=100-lr=3e5-bs=32-msl=384-seed=27/{}'

In [18]:
ALL_CHECKPOINTS = [1, 2, 3, 4, 5, 6, 8, 10, 12, 14, 16, 20, 24, 28, 32, 36, 44, 52, 60, 68, 76, 92, 108, 124, 140, 156, 172, 188, 204, 220, 236, 252, 268, 284, 300, 316, 332, 348, 364, 380, 396, 428, 460, 492, 524, 556, 588, 620, 652, 684, 716, 748, 780, 812, 844, 876, 908, 940, 972, 1004, 1036, 1100, 1164, 1228, 1292, 1356, 1420, 1484, 1548, 1612, 1676, 1804, 1932, 2060, 2188, 2316, 2444, 2572, 2700, 2828, 2956, 3084, 3212, 3340, 3468, 3596, 3724, 3852, 3980, 4108, 4236, 4364, 4492, 4620, 4748, 4876, 5004, 5132, 5260, 5388, 5516, 5644, 5772, 5900, 6028, 6156, 6284, 6412, 6540, 6668, 6796, 6924, 7052, 7180, 7308, 7436, 7564, 7692, 7820, 7948]
len(ALL_CHECKPOINTS)

120

SQuAD v2

In [4]:
for seed in [27, 29, 30]:
    chkpts = os.listdir(f'predictions/albert-xlarge-v2-squadv2-wu=100-lr=3e5-bs=32-msl=384-seed={seed}')
c
    for chkpt in chkpts:
        if len(os.listdir(f'predictions/albert-xlarge-v2-squadv2-wu=100-lr=3e5-bs=32-msl=384-seed={seed}/{chkpt}')) != 5:
            print(seed, chkpt)

In [4]:
SQ_ADQA_CHECKPOINTS = [1, 2, 3, 4, 5, 6, 8, 10, 12, 14, 16, 20, 24, 28, 32, 36, 44, 52, 60, 68, 76, 92, 108, 124, 140, 156, 172, 188, 204, 220, 236, 252, 268, 284, 300, 316, 332, 348, 364, 380, 396, 428, 460, 492, 524, 556, 588, 620, 652, 684, 716, 748, 780, 812]
len(SQ_ADQA_CHECKPOINTS)

54

In [8]:
remaining_chkpts_list = []

for seed in [27, 28, 29]:
    chkpts = os.listdir(f'predictions/on-train/albert-xlarge-v2-squadv1-adversarialall-wu=100-lr=3e5-bs=32-msl=384-seed={seed}')
    chkpts = [int(i.split('-')[-1]) for i in chkpts]
    
    remaining_chkpts = set(SQ_ADQA_CHECKPOINTS) - set(chkpts)
    remaining_chkpts_list.append(remaining_chkpts)
    
#     chkpts = np.sort(chkpts)
#     print(seed, chkpts[-1])
    print(seed, remaining_chkpts)

27 set()
28 set()
29 {1, 2, 3, 4, 5, 6, 8, 10, 12, 14, 20, 28, 32, 36, 44, 52, 68, 76, 92, 108, 124}


In [13]:
chkpts = os.listdir(f'predictions/on-train/albert-xlarge-v2-squadv1-adversarialall-wu=100-lr=3e5-bs=32-msl=384-seed=29')

for chkpt in chkpts:
    if len(os.listdir(f'predictions/on-train/albert-xlarge-v2-squadv1-adversarialall-wu=100-lr=3e5-bs=32-msl=384-seed=29/{chkpt}')) < 2:
        print(chkpt)

SQuAD v1

In [5]:
for model in ['dbert', 'dbidaf', 'droberta']:
    for seed in [28, 29, 30]:
        chkpts = os.listdir(f'predictions/albert-xlarge-v2-squadv1-wu=100-lr=3e5-bs=32-msl=384-seed={seed}-{model}')

        for chkpt in chkpts:
            if len(os.listdir(f'predictions/albert-xlarge-v2-squadv1-wu=100-lr=3e5-bs=32-msl=384-seed={seed}/{chkpt}')) != 4:
                print(model, seed, chkpt)

In [6]:
for model in ['dbert', 'dbidaf', 'droberta']:
    for seed in [28, 29, 30]:
        chkpts = os.listdir(f'predictions/albert-xlarge-v2-squadv1-wu=100-lr=3e5-bs=32-msl=384-seed={seed}-{model}')
        chkpts = [int(i.split('-')[-1]) for i in chkpts]

        remaining_chkpts = set(ALL_CHECKPOINTS) - set(chkpts)
        
        if len(remaining_chkpts) != 0:
            print(model, seed, remaining_chkpts)

In [16]:
remaining_chkpts_list

[{380, 6284, 6412, 6540, 6668, 6796, 6924, 7052, 7180, 7308, 7820, 7948},
 {380,
  6284,
  6412,
  6540,
  6668,
  6796,
  6924,
  7052,
  7180,
  7308,
  7564,
  7692,
  7820,
  7948},
 {6284, 6412, 6540, 6668, 6796, 6924, 7052, 7180, 7308, 7692, 7820, 7948},
 {6156, 6284, 6412, 6540, 6668, 6796, 6924, 7180, 7308, 7692, 7820, 7948},
 {6156, 6284, 6412, 6540, 6668, 6796, 6924, 7180, 7436, 7692, 7820, 7948},
 {6156, 6284, 6412, 6540, 6668, 6796, 6924, 7052, 7692, 7820, 7948}]

In [19]:
remaining_chkpts_list_union = list(set().union(*remaining_chkpts_list))
remaining_chkpts_list_union

[7948,
 6540,
 7052,
 6412,
 6924,
 6668,
 7180,
 6284,
 6796,
 7308,
 7436,
 7564,
 7692,
 7820,
 380,
 6156]

In [20]:
len(remaining_chkpts_list_union)

16

In [28]:
108 in remaining_chkpts_list_union

False

In [29]:
remaining_chkpts_list_union.append(108)

In [30]:
remaining_chkpts_list_union

[7948,
 6540,
 7052,
 6412,
 6924,
 6668,
 7180,
 6284,
 6796,
 7308,
 7436,
 7564,
 7692,
 7820,
 380,
 6156,
 108]

In [31]:
len(remaining_chkpts_list_union)  * 6

102

In [6]:
remaining_chkpts_list = []

for seed in [27, 28, 29]:
    chkpts = os.listdir(f'predictions/on-train/albert-xlarge-v2-squadv1-adversarialall-wu=100-lr=3e5-bs=32-msl=384-seed=27/checkpoint-1')
    chkpts = [int(i.split('-')[-1]) for i in chkpts]
    
    print(len(chkpts))

15
8
2


In [12]:
string = ""
for num in (set(ALL_CHECKPOINTS) - set(chkpts)):
    string += str(num)
    string += ', '

In [13]:
string

'2188, 6284, 2316, 2444, 3212, 4236, 3596, 7564, 6924, 2572, 3084, 6412, 2060, 7436, 2700, 2828, 6796, 7052, 2956, 7308, 3340, 3468, 3724, 7820, 7948, 3852, 3980, 4108, 4364, 6668, 4492, 4620, 4748, 4876, 5004, 5132, 5260, 5388, 5516, 5772, 5900, 6028, 6156, '

In [7]:
chkpts = os.listdir(f'predictions/checklist/albert-xlarge-v2-squadv1-adversarialall-wu=100-lr=3e5-bs=32-msl=384-seed=28')

for chkpt in chkpts:
    if len(os.listdir(f'predictions/checklist/albert-xlarge-v2-squadv1-adversarialall-wu=100-lr=3e5-bs=32-msl=384-seed=28/{chkpt}')) < 4:
        print(chkpt)

In [ ]:
60, 32, 68, 36, 44, 76, 92, 2572, 4748

seed 30 - checkpoint 16
seed 29 - 6028, 6412

In [5]:
import pandas as pd

In [13]:
remaining_chkpts_list = []
missing_checkpoints = []

for dataset in ['dbert', 'dbidaf', 'droberta', 'squad']:
    for seed in [27, 28, 29]:
        chkpts = os.listdir(f'predictions/albert-xlarge-v2-squadv1-adversarialall-wu=100-lr=3e5-bs=32-msl=384-seed={seed}-{dataset}')
        chkpts = [int(i.split('-')[-1]) for i in chkpts]
        
        missing = list(set(ALL_CHECKPOINTS) - set(chkpts))
        missing_checkpoints += missing
        
        print(f'{seed}-{dataset}')
        print(len(chkpts))
        print('\n')

27-dbert
71


28-dbert
71


29-dbert
71


27-dbidaf
71


28-dbidaf
71


29-dbidaf
71


27-droberta
71


28-droberta
71


29-droberta
71


27-squad
70


28-squad
70


29-squad
70




In [15]:
len(set(missing_checkpoints))

50

In [16]:
string = ""
for num in (set(missing_checkpoints)):
    string += str(num)
    string += ', '

In [17]:
string

'2956, 7052, 6412, 2444, 6540, 6284, 2316, 2188, 3212, 4236, 6924, 2700, 2828, 6796, 3084, 7180, 7308, 7436, 3340, 7564, 3468, 3596, 7692, 3724, 7820, 7948, 3852, 3980, 4108, 2572, 4364, 6668, 4492, 4620, 4748, 4876, 5004, 5132, 5260, 6028, 5388, 5516, 5644, 5772, 1676, 1804, 5900, 6156, 1932, 2060, '

In [12]:
string = ""
for num in (set(ALL_CHECKPOINTS) - set(chkpts)):
    string += str(num)
    string += ', '

In [13]:
string

'2188, 6284, 2316, 2444, 3212, 4236, 3596, 7564, 6924, 2572, 3084, 6412, 2060, 7436, 2700, 2828, 6796, 7052, 2956, 7308, 3340, 3468, 3724, 7820, 7948, 3852, 3980, 4108, 4364, 6668, 4492, 4620, 4748, 4876, 5004, 5132, 5260, 5388, 5516, 5772, 5900, 6028, 6156, '

In [4]:
remaining_chkpts_list = []

for seed in [27, 28, 29]:
    for model in ['dbert', 'dbidaf', 'droberta', 'squad']:
        chkpts = os.listdir(f'predictions/albert-xlarge-v2-squadv1-adversarialall-wu=100-lr=3e5-bs=32-msl=384-seed={seed}-{model}')
        chkpts = [int(i.split('-')[-1]) for i in chkpts]
    
        print(len(chkpts))

120
120
120
120
120
120
120
120
120
120
120
120


In [12]:
string = ""
for num in (set(ALL_CHECKPOINTS) - set(chkpts)):
    string += str(num)
    string += ', '

In [13]:
string

'2188, 6284, 2316, 2444, 3212, 4236, 3596, 7564, 6924, 2572, 3084, 6412, 2060, 7436, 2700, 2828, 6796, 7052, 2956, 7308, 3340, 3468, 3724, 7820, 7948, 3852, 3980, 4108, 4364, 6668, 4492, 4620, 4748, 4876, 5004, 5132, 5260, 5388, 5516, 5772, 5900, 6028, 6156, '

In [5]:
for seed in [27, 28, 29]:
    for model in ['dbert', 'dbidaf', 'droberta', 'squad']:
        chkpts = os.listdir(f'predictions/albert-xlarge-v2-squadv1-adversarialall-wu=100-lr=3e5-bs=32-msl=384-seed={seed}-{model}')

        for chkpt in chkpts:
            if len(os.listdir(f'predictions/albert-xlarge-v2-squadv1-adversarialall-wu=100-lr=3e5-bs=32-msl=384-seed={seed}-{model}/{chkpt}')) < 4:
                print(f'predictions/albert-xlarge-v2-squadv1-adversarialall-wu=100-lr=3e5-bs=32-msl=384-seed={seed}-{model}/{chkpt}')
                print('\n')

In [ ]:
1484, 1548, 1612

In [5]:
for seed in [27, 28, 29]:
    for chkpt in ALL_CHECKPOINTS:
        try:
            pd.read_csv(f'predictions/checklist/albert-xlarge-v2-squadv1-adversarialall-wu=100-lr=3e5-bs=32-msl=384-seed={seed}/checkpoint-{chkpt}/checklist_results_summary.txt')
        except FileNotFoundError:
            pass

        for chkpt in chkpts:
            if len(os.listdir(f'predictions/albert-xlarge-v2-squadv1-adversarialall-wu=100-lr=3e5-bs=32-msl=384-seed={seed}-{model}/{chkpt}')) < 4:
                print(f'predictions/albert-xlarge-v2-squadv1-adversarialall-wu=100-lr=3e5-bs=32-msl=384-seed={seed}-{model}/{chkpt}')
                print('\n')

In [3]:
import argparse
import re
import os

In [5]:
os.getcwd()

'/SAN/intelsys/rclearn/when-do-reading-comprehension-models-learn/notebooks/exploration'

In [38]:
for seed in [28]:  # [27, 28, 29]:
    for chkpt in [8]:  # ALL_CHECKPOINTS:
        with open(f'predictions/checklist/albert-xlarge-v2-squadv1-wu=100-lr=3e5-bs=32-msl=384-seed={seed}/checkpoint-{chkpt}/checklist_results.txt', 'r') as f:
            summary = f.read()

        contexts = re.findall(r"[\n\r].*C:\s*([^\n\r]*)", summary)
        preds = re.findall(r"[\n\r].*P:\s*([^\n\r]*)", summary)
        assert len(contexts) == len(preds)

        count = 0
        affected_checkpoints = []
        for c, p in zip(contexts, preds):
            if p =='empty':
                continue
            else:
                try:
                    assert p in c
#                     assert '\n' not in p
                except:
                    count += 1 
                    print("Context:", c)
                    print("P:", p)
                    print('\n')

        if count > 0:
            print(seed, chkpt, '-', count)

Context: Tesla gained experience in telephony and electrical engineering before emigrating to the United States in 1884 to work for Thomas Edison in New York City. He soon struck out on his own with financial backers, setting up laboratories and companies to develop a range of electrical devices. His patented AC induction motor and transformer were licensed by George Westinghouse, who also hired Tesla for a short time as a consultant. His work in the formative years of electric power development was involved in a corporate alternating current/direct current "War of Currents" as well as various patent battles.
P: as they have already been eliminated from consideration after


Context: Tesla gained experience in telephony and electrical engineering before emigrating to the United States in 1884 to work for Thomas Edison in New York City. He soon struck out on his own with financial backers, setting up laboratories and companies to develop a range of electrical devices. His patented AC in

In [30]:
contexts

['Kevin is greater than Jennifer.',
 'Kevin is greater than Jennifer.',
 'Andrew is slower than Timothy.',
 'Andrew is slower than Timothy.',
 'Sara is darker than William.',
 'Sara is darker than William.',
 'Benjamin is highly pessimistic about the project. Sarah is pessimistic about the project.',
 'Benjamin is highly pessimistic about the project. Sarah is pessimistic about the project.',
 'Sarah is pessimistic about the project. Benjamin is highly pessimistic about the project.',
 'Jessica is highly happy about the project. Jacob is happy about the project.',
 'Jessica is highly happy about the project. Jacob is happy about the project.',
 'Jacob is happy about the project. Jessica is highly happy about the project.',
 'Lisa is very cautious about the project. Joseph is cautious about the project.',
 'Lisa is very cautious about the project. Joseph is cautious about the project.',
 'Joseph is cautious about the project. Lisa is very cautious about the project.',
 'There is a round

In [24]:
len(contexts), len(preds)

(149, 151)

In [25]:
preds

['Emily is older than Nicholas',
 'Michael is older than Natalie',
 'Lauren is greater than Richard',
 'Justin',
 'Emily',
 'Emily is enthusiastic about the project. Justin',
 'Abigail is really positive about the project. Jordan',
 'Abigail is really positive about the project. Jordan',
 'Abigail',
 'Laura',
 'red',
 'new blue',
 'new and blue',
 'old and blue',
 'American executive',
 'Japanese architect',
 'Bangladeshi auditor',
 'tractor',
 'tractor',
 'tractor',
 'Jeffrey bought a tractor. Jose',
 'Laura bought a snake. Kimberly',
 'Kimberly',
 'Emily',
 'Emily',
 'Jennifer is worse than Victoria',
 'Aaron is smarter than Daniel',
 'Daniel is dumber than Aaron',
 'Amy is better than Ryan',
 'Ryan is worse than Amy',
 'Sarah is more visible than Kelly',
 'Sarah is more visible than Kelly',
 'Sarah is more visible than Kelly',
 'Eric is more unhappy than Danielle',
 'Danielle',
 'Eric',
 'Jacob is more cautious than Ryan',
 'Ryan',
 'Ryan',
 '22',
 'chorale cantatas',
 'immunization